# Create Heterostructure Example with Three Materials

This notebook demonstrates how to create a heterostructure involving three different materials using a sequential interface creation approach. We first create an interface between **Material 0** and **Material 1**, and then use that interface as a substrate to add a film of **Material 2**.

<h2 style="color:green">Usage</h2>

1. **Set up the notebook and install packages**
2. **Import materials from Standata**
3. **Select and preview materials for the heterostructure**
4. **Build the heterostructure layer by layer with ZSL interface builder**

## Summary

1. **Prepare the Environment:** Set up the notebook and install packages, preview the input materials.
2. **Create Interfaces:** Sequentially create interfaces between the materials.
3. **Visualize:** Preview the materials and resulting interfaces.
4. **Pass to Runtime:** Pass the final heterostructure to the external runtime.

## Notes

1. For more information, see [Introduction](Introduction.ipynb)

<!-- # TODO: use a hashtag-based anchor link to interface creation documentation above -->


## 1. Prepare the Environment
### 1.1. Set up the Notebook

Set the following flags to control the notebook behavior.


In [ ]:
# Enable interactive selection of terminations via UI prompt
IS_TERMINATIONS_SELECTION_INTERACTIVE = False 

# Indices and configurations for the three materials
MATERIAL_0_INDEX = 0
MATERIAL_1_INDEX = 1
MATERIAL_2_INDEX = 2

# Interface parameters
MAX_AREA_01 = 50 # search area for the first interface
MAX_AREA_12 = 200 # search area for the second interface
INTERFACE_01_DISTANCE = 3.0  # in Angstrom
INTERFACE_12_DISTANCE = 3.0  # in Angstrom
FINAL_INTERFACE_VACUUM = 20.0  # in Angstrom

# Configuration for Material 0 (Substrate)
MATERIAL_0_MILLER_INDICES = (0, 0, 1)
MATERIAL_0_THICKNESS = 3  # in atomic layers
MATERIAL_0_VACUUM = 3  # in Angstroms
MATERIAL_0_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
MATERIAL_0_USE_ORTHOGONAL_C = True

# Configuration for Material 1 (Film 1)
MATERIAL_1_MILLER_INDICES = (0, 0, 1)
MATERIAL_1_THICKNESS = 1  # in atomic layers
MATERIAL_1_VACUUM = 0  # in Angstroms
MATERIAL_1_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
MATERIAL_1_USE_ORTHOGONAL_C = True

# Configuration for Material 2 (Film 2)
MATERIAL_2_MILLER_INDICES = (0, 0, 1)
MATERIAL_2_THICKNESS = 1  # in atomic layers
MATERIAL_2_VACUUM = 1  # in Angstroms
MATERIAL_2_XY_SUPERCELL_MATRIX = [[1, 0], [0, 1]]
MATERIAL_2_USE_ORTHOGONAL_C = True

# Set termination pair indices for both interfaces
TERMINATION_PAIR_INDEX_01 = 0
TERMINATION_PAIR_INDEX_12 = 0


### 1.2. Install Packages

The step executes only in Pyodide environment. For other environments, the packages should be installed via `pip install` (see [README](../../README.ipynb)).


In [ ]:
import sys

if sys.platform == "emscripten":
    import micropip
    await micropip.install("https://exabyte-io.github.io/api-examples/mat3ra_api_examples-0.1.dev1+gc41b0b8-py3-none-any.whl", deps=False)
    await micropip.install('mat3ra-utils')
    from mat3ra.utils.jupyterlite.packages import install_packages
    await install_packages("create_interface_with_min_strain_zsl.ipynb")


### 1.3. Get Input Materials and Assign `material0`, `material1`, and `material2`

Materials are loaded with `get_materials()`. The first material is assigned as **Material 0**, the second as **Material 1**, and the third as **Material 2**.


In [ ]:
from utils.jupyterlite import get_materials

materials = get_materials(globals())

material0 = materials[MATERIAL_0_INDEX]

try: 
    material1 = materials[MATERIAL_1_INDEX]
except IndexError:
    print("Please select Material 1. Material 1 is set to Material 0.")
    material1 = material0

try:
    material2 = materials[MATERIAL_2_INDEX]
except IndexError:
    print("Please select Material 2. Material 2 is set to Material 0.")
    material2 = material0


### 1.4. Preview Original Materials

Visualize the three original materials.


In [ ]:
from utils.visualize import visualize_materials as visualize

visualize([material0, material1, material2], repetitions=[3, 3, 1], rotation="0x")


## 2. Create First Interface (Material 0 + Material 1)

### 2.1. Configure Slabs and Select Termination Pair

Set up slab configurations for **Material 0** and **Material 1**, then select terminations for the first interface.


In [ ]:
from mat3ra.made.tools.build.slab.helpers import get_slab_terminations
from mat3ra.made.tools.build.slab.configurations import SlabConfiguration

# Slab Configuration for Material 1 (Film) - using correct parameters like ZSL notebook
material1_slab_configuration = SlabConfiguration.from_parameters(
    material_or_dict=material1,
    miller_indices=MATERIAL_1_MILLER_INDICES,
    number_of_layers=MATERIAL_1_THICKNESS, # in atomic layers
    vacuum=MATERIAL_1_VACUUM, # in Angstroms
    termination_formula=None,  # if None, the first termination will be used
    use_conventional_cell=True
)

# Slab Configuration for Material 0 (Substrate) - using correct parameters like ZSL notebook
material0_slab_configuration = SlabConfiguration.from_parameters(
    material_or_dict=material0,
    miller_indices=MATERIAL_0_MILLER_INDICES,
    number_of_layers=MATERIAL_0_THICKNESS, # in atomic layers
    vacuum=MATERIAL_0_VACUUM, # in Angstroms
    termination_formula=None,  # if None, the first termination will be used
    use_conventional_cell=True
)

# Get possible terminations for the slabs
material1_slab_terminations = get_slab_terminations(material=material1, miller_indices=MATERIAL_1_MILLER_INDICES)
material0_slab_terminations = get_slab_terminations(material=material0, miller_indices=MATERIAL_0_MILLER_INDICES)

# Visualize all possible terminations using analyzer approach
from mat3ra.made.tools.analyze.lattice_planes import CrystalLatticePlanesMaterialAnalyzer

material1_analyzer = CrystalLatticePlanesMaterialAnalyzer(material=material1, miller_indices=MATERIAL_1_MILLER_INDICES)
material1_slabs = [material1_analyzer.get_material_with_termination_without_vacuum(termination) for termination in material1_slab_terminations]

material0_analyzer = CrystalLatticePlanesMaterialAnalyzer(material=material0, miller_indices=MATERIAL_0_MILLER_INDICES)
material0_slabs = [material0_analyzer.get_material_with_termination_without_vacuum(termination) for termination in material0_slab_terminations]

material1_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in zip(material1_slabs, material1_slab_terminations)]
material0_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in zip(material0_slabs, material0_slab_terminations)]

visualize(material1_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")
visualize(material0_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")


### 2.2. Print and Select Termination Pair for First Interface


In [ ]:
from itertools import product

termination_pairs_01 = list(product(material1_slab_terminations, material0_slab_terminations))    
print("Termination Pairs for First Interface (Material1, Material0)")
for idx, termination_pair in enumerate(termination_pairs_01):
    print(f"    {idx}: {termination_pair}")


### 2.3. Select Termination Pair for First Interface


In [ ]:
from mat3ra.made.tools.build.slab.termination_utils import select_slab_termination
from utils.io import ui_prompt_select_array_element_by_index, ui_prompt_select_array_element_by_index_pyodide

# Select terminations for each material using the newer approach
material1_termination = select_slab_termination(material1_slab_terminations, None)  # None means use first termination
material0_termination = select_slab_termination(material0_slab_terminations, None)  # None means use first termination

# Create termination pair
termination_pair_first = (material1_termination, material0_termination)

# Allow for interactive selection if enabled
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    termination_pair_index_01 = TERMINATION_PAIR_INDEX_01
    termination_pair_first = termination_pairs_01[termination_pair_index_01]
    if sys.platform == "emscripten":
        termination_pair_first = await ui_prompt_select_array_element_by_index_pyodide(
            termination_pairs_01,
            element_name="Material1/Material0 termination pair"
        )
    else:
        termination_pair_first = ui_prompt_select_array_element_by_index(
            termination_pairs_01,
            element_name="Material1/Material0 termination pair"
        )


### 2.4. Initialize Interface Configuration for First Interface


In [ ]:
from mat3ra.made.tools.build.slab.builders import SlabBuilder

# Create actual slabs using the newer approach
material1_termination, material0_termination = termination_pair_first

# Update configurations with selected terminations - using correct parameters like ZSL notebook
material1_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=material1,
    miller_indices=MATERIAL_1_MILLER_INDICES,
    number_of_layers=MATERIAL_1_THICKNESS,
    vacuum=0.0,  # Set vacuum to 0 for interface creation
    termination_formula=None,  # Will use selected termination
    use_conventional_cell=True
)

material0_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=material0,
    miller_indices=MATERIAL_0_MILLER_INDICES,
    number_of_layers=MATERIAL_0_THICKNESS,
    vacuum=0.0,  # Set vacuum to 0 for interface creation
    termination_formula=None,  # Will use selected termination
    use_conventional_cell=True
)

# Build the actual slabs
material1_slab = SlabBuilder().get_material(material1_slab_config)
material0_slab = SlabBuilder().get_material(material0_slab_config)


### 2.5. Set Strain Matching Parameters and Generate First Interface


In [ ]:
from mat3ra.made.tools.analyze.interface import ZSLInterfaceAnalyzer

# Set up ZSL Interface Analyzer with the newer approach
zsl_analyzer_01 = ZSLInterfaceAnalyzer(
    substrate_slab_configuration=material0_slab_config,
    film_slab_configuration=material1_slab_config,
    max_area=MAX_AREA_01,
    max_area_ratio_tol=0.09,  # Default tolerance
    max_angle_tol=0.03,       # Default tolerance
    max_length_tol=0.03       # Default tolerance
)

# Get ZSL matches
matches_01 = zsl_analyzer_01.zsl_match_holders


### 2.6. Plot and Select First Interface


In [ ]:
from utils.plot import plot_strain_vs_area
from mat3ra.made.tools.build.interface.helpers import create_zsl_interface_between_slabs

PLOT_SETTINGS = {
    "HEIGHT": 600,
    "X_SCALE": "log",  # or linear
    "Y_SCALE": "log",  # or linear
}

plot_strain_vs_area(matches_01, PLOT_SETTINGS)

# Select the interface with the lowest strain and smallest number of atoms
selected_index_01 = 0

# Create the first interface using the newer approach
interface_01 = create_zsl_interface_between_slabs(
    substrate_slab=material0_slab,
    film_slab=material1_slab,
    gap=INTERFACE_01_DISTANCE,
    vacuum=FINAL_INTERFACE_VACUUM,
    match_id=selected_index_01,
    max_area=MAX_AREA_01,
)

selected_interfaces_01 = [interface_01]


### 2.7. Preview the First Interface


In [ ]:
visualize(selected_interfaces_01, repetitions=[3, 3, 1])
visualize(selected_interfaces_01, repetitions=[3, 3, 1], rotation="-90x")

## 3. Create Second Interface (First Interface + Material 2)

### 3.1. Configure Slabs and Select Termination Pair for Second Interface

Now, use the first interface as the substrate to add **Material 2**.


In [ ]:
from mat3ra.made.tools.modify import translate_to_z_level

# Update substrate to be the first interface
substrate_second = translate_to_z_level(selected_interfaces_01[0], "top")

# Get possible terminations for Material 2 and the substrate (first interface)
material2_slab_terminations = get_slab_terminations(material=material2, miller_indices=MATERIAL_2_MILLER_INDICES)
# For the substrate (first interface), we'll use a simple approach since it's already an interface
substrate_second_slab_terminations = ["top"]  # Simple termination for the interface substrate

# Visualize Material 2 terminations using analyzer approach
material2_analyzer = CrystalLatticePlanesMaterialAnalyzer(material=material2, miller_indices=MATERIAL_2_MILLER_INDICES)
material2_slabs = [material2_analyzer.get_material_with_termination_without_vacuum(termination) for termination in material2_slab_terminations]

material2_slabs_with_titles = [{"material": slab, "title": str(termination)} for slab, termination in zip(material2_slabs, material2_slab_terminations)]

visualize(material2_slabs_with_titles, repetitions=[3, 3, 1], rotation="-90x")

# Visualize the substrate (first interface)
visualize([{"material": substrate_second, "title": "First Interface (Substrate)"}], repetitions=[3, 3, 1], rotation="-90x")

### 3.2. Print and Select Termination Pair for Second Interface


In [ ]:
termination_pairs_12 = list(product(material2_slab_terminations, substrate_second_slab_terminations))    
print("Termination Pairs for Second Interface (Material2, First Interface Substrate)")
for idx, termination_pair in enumerate(termination_pairs_12):
    print(f"    {idx}: {termination_pair}")

### 3.3. Select Termination Pair for Second Interface


In [ ]:
# Select terminations for Material 2 and the substrate using the newer approach
material2_termination = select_slab_termination(material2_slab_terminations, None)  # None means use first termination
substrate_second_termination = substrate_second_slab_terminations[0]  # Use the first (and only) termination

# Create termination pair
termination_pair_second = (material2_termination, substrate_second_termination)

# Allow for interactive selection if enabled
if IS_TERMINATIONS_SELECTION_INTERACTIVE:
    termination_pair_index_12 = TERMINATION_PAIR_INDEX_12
    termination_pair_second = termination_pairs_12[termination_pair_index_12]
    if sys.platform == "emscripten":
        termination_pair_second = await ui_prompt_select_array_element_by_index_pyodide(
            termination_pairs_12,
            element_name="Material2/First Interface termination pair"
        )
    else:
        termination_pair_second = ui_prompt_select_array_element_by_index(
            termination_pairs_12,
            element_name="Material2/First Interface termination pair"
        )

### 3.4. Initialize Interface Configuration for Second Interface


In [ ]:
# Create slab configurations and slabs for the second interface using the newer approach
material2_termination, substrate_second_termination = termination_pair_second

# Slab configuration for Material 2 - using correct parameters like ZSL notebook
material2_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=material2,
    miller_indices=MATERIAL_2_MILLER_INDICES,
    number_of_layers=MATERIAL_2_THICKNESS,
    vacuum=0.0,  # Set vacuum to 0 for interface creation
    termination_formula=None,  # Will use selected termination
    use_conventional_cell=True
)

# Build the Material 2 slab
material2_slab = SlabBuilder().get_material(material2_slab_config)

# For the substrate (first interface), we'll use it as-is since it's already built

### 3.5. Set Strain Matching Parameters and Generate Second Interface


In [ ]:
# Set up ZSL Interface Analyzer for the second interface
# Note: For the second interface, we need to create a slab configuration for the substrate (first interface)
substrate_second_slab_config = SlabConfiguration.from_parameters(
    material_or_dict=substrate_second,
    miller_indices=(0, 0, 1),  # Z-orientation for the first interface
    number_of_layers=1,  # One unit cell thick
    vacuum=0.0,
    termination_formula=None,
    use_conventional_cell=True
)

zsl_analyzer_12 = ZSLInterfaceAnalyzer(
    substrate_slab_configuration=substrate_second_slab_config,
    film_slab_configuration=material2_slab_config,
    max_area=MAX_AREA_12,
    max_area_ratio_tol=0.09,  # Default tolerance
    max_angle_tol=0.03,       # Default tolerance
    max_length_tol=0.03       # Default tolerance
)

# Get ZSL matches for the second interface
matches_12 = zsl_analyzer_12.zsl_match_holders

### 3.6. Plot and Select Second Interface


In [ ]:
plot_strain_vs_area(matches_12, PLOT_SETTINGS)

# Select the interface with the lowest strain and smallest number of atoms
selected_index_12 = 0

# Create the second interface using the newer approach
interface_12 = create_zsl_interface_between_slabs(
    substrate_slab=substrate_second,
    film_slab=material2_slab,
    gap=INTERFACE_12_DISTANCE,
    vacuum=FINAL_INTERFACE_VACUUM,
    match_id=selected_index_12,
    max_area=MAX_AREA_12,
)

selected_interfaces_12 = [interface_12]

### 3.7. Preview the Second Interface (Final Heterostructure)


In [ ]:
visualize(selected_interfaces_12, repetitions=[3, 3, 1])
visualize(selected_interfaces_12, repetitions=[3, 3, 1], rotation="-90x")

## 4. Preview the Final Heterostructure


In [ ]:
visualize(selected_interfaces_12, repetitions=[3, 3, 1], title="Final Heterostructure (First Interface + Material2)")
visualize(selected_interfaces_12, repetitions=[3, 3, 1], rotation="-90x", title="Final Heterostructure (First Interface + Material2) Rotated")

## 5. Pass the Final Heterostructure to the Outside Runtime

Pass the resulting heterostructure with an adjusted name to `set_materials()`.


In [ ]:
from utils.jupyterlite import set_materials

final_heterostructure = selected_interfaces_12[0]
final_heterostructure.name = f"{material0.name} - {material1.name} - {material2.name} - Heterostructure"

set_materials(final_heterostructure)